# Sistemas inteligentes para respostas a perguntas médicas

Gyovana M. Moriyama (216190)

Rafael A. Matumoto (273085)

In [1]:
!pip install -qU langchain_openai langchain-community faiss-cpu sentence-transformers openai datasets pydantic langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import os
import datasets
import datetime
import re
import pandas as pd
import numpy as np
from pprint import pprint
from google.colab import userdata, drive

from tqdm import tqdm
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_core.load import dumps, loads

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
os.environ['OPENAI_API_KEY'] = userdata.get('api_key')

In [5]:
folder_path = '/content/drive/MyDrive/ia024a/projeto/entrega4/'
# folder_path = '/content/drive/MyDrive/IA024A_Processamento_de_Linguagem_Natural/Projeto Final/projeto/entrega4/'

## 1 Data processing

Google drive files structure

```
data_clean/
|-- questions/
|   |-- Mainland/
|   |-- Taiwan/
|   |-- US/
|       |-- 4_options/
|       |   |-- phrases_no_exclude_dev.jsonl
|       |   |-- phrases_no_exclude_test.jsonl
|       |   |-- phrases_no_exclude_train.jsonl
|       |-- dev.jsonl
|       |-- test.jsonl
|       |-- train.jsonl
|       |-- ...
|-- textbooks/
|   |-- en/
|   |   |-- Anatomy_Gray.txt
|   |   |-- ...
|   |-- zh_paragraph/
|   |-- zh_sentence/
```

### 1.1 Load datasets

#### 1.1.1 MedQA-USMLE-4-options dataset

In [6]:
# run experiments on train split
train_data = datasets.load_dataset('GBaker/MedQA-USMLE-4-options', split='train')

# evaluate on test split
test_data = datasets.load_dataset('GBaker/MedQA-USMLE-4-options', split='test')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

phrases_no_exclude_train.jsonl:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

phrases_no_exclude_test.jsonl:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [ ]:
# add a column with indexes (original question order)
train_data = train_data.add_column('original_quest_id', range(len(train_data)))

In [ ]:
# sample questions for experiments
sampled_data = test_data.shuffle(seed=42).select(range(2))

#### 1.1.2 MedQA textbooks

In [ ]:
# download MedQA data from https://drive.google.com/file/d/1ImYUSLk9JbgHXOemfvyiDiirluZHPeQw/view?usp=sharing
!gdown -q 1ImYUSLk9JbgHXOemfvyiDiirluZHPeQw
!unzip -q data_clean.zip -d /content/medQA

In [ ]:
def load_documents():
    '''
    Loads reference textbooks for MedQA as a list of Document objects.
    '''

    medqa_path = '/content/medQA/data_clean/textbooks/en/'
    loader = DirectoryLoader(medqa_path, glob='**/*.txt', loader_cls=TextLoader)

    # list of 'Document' files, with 'source'/file path as metadata
    documents = loader.load()

    return documents

### 1.2 Exploratory analysis

#### 1.2.1 MedQA questions

In [ ]:
# load test data as a Pandas dataframe
test_data_df = test_data.to_pandas()

In [ ]:
test_data_df.head()

,question,answer,options,meta_info,answer_idx,metamap_phrases
0,A junior orthopaedic surgery resident is compl...,Tell the attending that he cannot fail to disc...,{'A': 'Disclose the error to the patient and p...,step1,B,"[junior orthopaedic surgery resident, completi..."
1,A 67-year-old man with transitional cell carci...,Cross-linking of DNA,"{'A': 'Inhibition of proteasome', 'B': 'Hypers...",step1,D,"[67 year old man, transitional cell carcinoma ..."
2,Two weeks after undergoing an emergency cardia...,Cholesterol embolization,"{'A': 'Renal papillary necrosis', 'B': 'Choles...",step2&3,B,"[Two weeks, emergency cardiac, stenting, unsta..."
3,A 39-year-old woman is brought to the emergenc...,"Lactose-fermenting, gram-negative rods forming...","{'A': 'Coagulase-positive, gram-positive cocci...",step1,D,"[year old woman, brought, emergency department..."
4,A 35-year-old man comes to the physician becau...,Ketotifen eye drops,"{'A': 'Erythromycin ointment', 'B': 'Ketotifen...",step2&3,B,"[35 year old man, physician, of itchy, watery,..."


In [ ]:
# number of questions by exam step
test_data_df.groupby('meta_info').size()

,0
meta_info,
step1,679
step2&3,594


In [ ]:
# distribution of answer alternatives
test_data_df.groupby(['meta_info', 'answer_idx']).size().reset_index()

,meta_info,answer_idx,0
0,step1,A,187
1,step1,B,156
2,step1,C,188
3,step1,D,148
4,step2&3,A,166
5,step2&3,B,153
6,step2&3,C,158
7,step2&3,D,117


In [ ]:
# join question and answer alternatives (must be included in the prompt)
test_data_df['quest_ans'] = test_data_df.apply(lambda x: x['question'] + str(x['options']), axis=1)

# question + answer alternatives length in number of characters (must fit in context size)
test_data_df['quest_ans'].str.len().describe()

,quest_ans
count,1273.000000
mean,888.602514
std,309.890528
min,230.000000
25%,679.000000
50%,852.000000
75%,1053.000000
max,3673.000000


#### 1.2.1 Reference textbooks

In [ ]:
# load textbooks as Document objects
docs = load_documents()

In [ ]:
def textbook_statistics(docs):
    '''
    Computes statistics for a list of textbooks.
    '''

    titles = list()
    total_num_chars = list()
    total_num_paragraphs = list()
    avg_num_chars_per_paragraph = list()
    avg_num_words_per_paragraph = list()
    max_paragraph_char_len = list()
    min_paragraph_char_len = list()

    for content in docs:
        titles.append(content.metadata['source'].split('/')[-1])
        total_num_chars.append(len(content.page_content))
        total_num_paragraphs.append(len(content.page_content.split('\n\n')))
        avg_num_chars_per_paragraph.append(np.mean([len(line) for line in content.page_content.split('\n\n')], dtype=int))
        avg_num_words_per_paragraph.append(np.mean([len(line.split()) for line in content.page_content.split('\n\n')], dtype=int))
        max_paragraph_char_len.append(max([len(line) for line in content.page_content.split('\n\n')]))
        min_paragraph_char_len.append(min([len(line) for line in content.page_content.split('\n\n')]))

    df = pd.DataFrame({
        'title': titles,
        'num_chars': total_num_chars,
        'num_paragraphs': total_num_paragraphs,
        'avg_num_chars_per_paragraph': avg_num_chars_per_paragraph,
        'avg_num_words_per_paragraph': avg_num_words_per_paragraph,
        'max_paragraph_char_len': max_paragraph_char_len,
        'min_paragraph_char_len': min_paragraph_char_len
    })

    return df

In [ ]:
textbook_statistics(docs)

,title,num_chars,num_paragraphs,avg_num_chars_per_paragraph,avg_num_words_per_paragraph,max_paragraph_char_len,min_paragraph_char_len
0,First_Aid_Step2.txt,1030582,6659,152,21,2471,2
1,Physiology_Levy.txt,3049236,8859,342,46,3779,2
2,Cell_Biology_Alberts.txt,4868257,10627,456,71,7671,2
3,Psichiatry_DSM-5.txt,2893358,12861,222,32,2932,2
4,Pharmacology_Katzung.txt,5122267,12869,396,56,5150,2
5,Pathoma_Husain.txt,399834,5306,73,10,733,2
6,Pediatrics_Nelson.txt,3002297,8034,371,52,4691,2
7,Histology_Ross.txt,3047020,6690,453,69,5169,2
8,Neurology_Adams.txt,8365987,16804,495,75,3348,2
9,Obstentrics_Williams.txt,6583361,21803,299,43,4981,2


## 2 Baseline evaluations



In [7]:
# model parameters
model = 'gpt-4o-mini'
model_temp = 0.5

In [8]:
# model
llm = ChatOpenAI(
    model=model,
    temperature=model_temp,
)

não tenho certeza se função funciona para todos os casos que temos

In [9]:
def run_model(model_chain, data, model_prompt, out_path, info=''):
    '''
    Invokes the model with ˋdataˋ and returns a text file containing raw outputs.
    '''

    filename = out_path + info + '_' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.txt'

    with open(filename, 'w') as f:

        f.write(''.center(10, '-'))
        f.write('\n')
        f.write(f'Prompt: {model_prompt}\n')
        f.write(''.center(10, '-'))
        f.write('\n[{')

        for n, question in enumerate(tqdm(data)):

            res = model_chain.invoke({'question': question['question'], 'options': question['options']})
            f.write(f'"{n}": ')

            # serialize results as a string
            f.write(dumps(res))
            f.write(',')

        f.write('"placeholder": "placeholder"') # add a placeholder k:v pair for trailling comma above
        f.write('}]')

    print(f'Results exported to {filename}')

In [10]:
def parse_answers_from_raw_output(raw_output):
    '''
    Parses answers from model contained in the raw output.
    '''

    chunk_size = raw_output.split('/')[-1].split('_')[2]
    chunk_overlap = raw_output.split('/')[-1].split('_')[3]
    reasoning = list()

    with open(raw_output) as f:
        # load file with LLM results
        tmp = f.readlines()

    try:
        # recover results as a dictionary
        recovered_results = loads(tmp[-1])[0]

        limit_error = list()
        not_parsed = list()
        answers = list()

        # matches literals A, B, C, or D at the beginning of a sentence (followed by the text of the corresponding option)
        # e.g. A (answer text) => A
        # other cases will be ignored
        ans_ptn = re.compile(r'(^[A|B|C|D])\s\(')

        for n, (k, v) in enumerate(recovered_results.items()):

            # skip placeholder key
            if k == 'placeholder':
                continue

            # max_iteration set to 5
            if v['output'] == 'Agent stopped due to iteration limit or time limit.':
                answers.append(None)
                limit_error.append(k)
                print(k, v['output'])

            elif len(v['output']) > 1:
                tmp_output = ans_ptn.findall(v['output'])
                if len(tmp_output) == 1:
                    answers.append(tmp_output[0])
                elif len(tmp_output) > 1:
                    answers.append(None)
                    not_parsed.append(k)
                    print(k, tmp_output)
                else:
                    answers.append(None)
                    not_parsed.append(k)
                    print(k, v['output'])

            else:
                if v['output'] in ['A', 'B', 'C', 'D']:
                    answers.append(v['output'])
                else:
                    answers.append(None)

            reasoning.append('\n'.join([i[0].log + i[1] for i in v['intermediate_steps']]))

    # loads function cannot deserialize structured outputs
    # regex pattern will extract the answers from the dictionary
    except NotImplementedError:
        # load string with raw output
        recovered_results = tmp[-1]
        # pprint(recovered_results)
        # regex pattern to get (question_number, question_answer_literal) pairs
        # parsed_pattern = re.compile(r'\"(\d+)\"\:\s\{\"raw\"\:.*?,\s\"parsed\".*?\"repr\"\:\s\"(Answer|AnswerCoT)\(final_answer\=\'([ABCD])\'')
        # parsed_pattern = re.compile(r'\"(\d+)\"\:\s\{\"raw\"\:.*?,\s\"parsed\".*?\"repr\"\:\s\"(Answer|AnswerCoT)\(final_answer\=\'([ABCD])(\'\,\sstep_by_step\=\'){0,1}(.*?)\'\)')
        parsed_pattern = re.compile(r'\"(\d+)\"\:\s\{\"raw\"\:.*?,\s\"parsed\".*?\"repr\"\:\s\"(Answer|AnswerCoT)\(final_answer\=\'([ABCD])(\,\sstep_by_step\=\'){0,1}(.*?)\)')

        parsed_answers = parsed_pattern.findall(recovered_results)
        answers = [i[2] for i in parsed_answers]
        print(len(parsed_answers))
        reasoning = [i[4] if i[1] == 'AnswerCoT' else '' for i in parsed_answers]

    return answers, reasoning

In [11]:
# @title Descomentar linha que salva csv com resultados
def accuracy_report(answers, dataset, out_path, file_name, reasoning=''):
    '''
    Calculates the accuracy of the answers.
    '''

    data2 = dataset.add_column('predictions', answers)
    if reasoning != '':
        data2 = data2.add_column('reasoning', reasoning)

    data2_df = data2.to_pandas()
    data2_df.fillna('unanswered', inplace=True)

    # save results to file
    data2_df.to_csv(out_path + file_name)

    # accuracy for all questions
    accuracy = data2_df['answer_idx'].eq(data2_df['predictions']).mean()

    # accuracy for step1 questions
    step1 = data2_df[data2_df['meta_info'].eq('step1')].copy()
    step1_accuracy = step1['answer_idx'].eq(step1['predictions']).mean()

    # accuracy for step2&3 questions
    step23 = data2_df[data2_df['meta_info'].eq('step2&3')].copy()
    step23_accuracy = step23['answer_idx'].eq(step23['predictions']).mean()

    accs = pd.DataFrame({
        'questions': ['general', 'step1', 'step2&3'],
        'accuracy': [accuracy, step1_accuracy, step23_accuracy]
    })

    print('Accuracy')
    print(accs.to_markdown())
    print()

    # distribution of wrong answers
    wrong = data2_df[~data2_df['answer_idx'].eq(data2_df['predictions'])].groupby('predictions').size().reset_index()

    print('Distribution of wrong answers')
    print(wrong.to_markdown())

### 2.1 Random guessing

In [ ]:
def guessing_accuracy(df, step=None):
    '''
    Calculates the accuracy of guessing the answer randomly.
    df (Dataframe): Dataframe with the questions
    step (str): Step of the question (step1, step2&3)

    Returns:
    (float, float): Mean and standard deviation of the accuracy
    '''

    res = list()
    rng = np.random.default_rng(42)

    if step is None:
        # calculates for the whole dataset
        step_data = df.copy()
    else:
        # calculates for the given step
        step_data = df[df['meta_info'].eq(step)].copy()

    # makes the random guess 100 times
    for _ in range(100):
        # gets a random answer from A to D for all questions and calculates the accuracy
        step_data['random_guess'] = np.random.choice(['A', 'B', 'C', 'D'], size=len(step_data))
        res.append(step_data['answer_idx'].eq(step_data['random_guess']).mean())

    res = np.array(res)
    return res.mean(), res.std()

In [ ]:
# random guessing accuracy
guessing_accuracy(test_data_df)

(0.25025137470542025, 0.01290652481258927)

In [ ]:
# random guessing accuracy in step1 questions
guessing_accuracy(test_data_df, step='step1')

(0.253181148748159, 0.017331124117086132)

In [ ]:
# random guessing accuracy in step2&3 questions
guessing_accuracy(test_data_df, step='step2&3')

(0.2528451178451179, 0.016442727417480787)

In [ ]:
# question length in number of characters
test_data_df['question'].str.len().describe()

,question
count,1273.000000
mean,745.579733
std,304.233918
min,67.000000
25%,536.000000
50%,715.000000
75%,909.000000
max,3543.000000


### 2.2 Direct question

In [ ]:
class Answer(BaseModel):
    '''
    Final answer to the question.
    '''
    final_answer: Literal['A', 'B', 'C', 'D'] = Field(description='Final answer to the question')

In [ ]:
direct_prompt_template = '''
question: {question}
options: {options}
Among A through D, the answer is:
'''.strip()

In [ ]:
direct_prompt = ChatPromptTemplate.from_template(direct_prompt_template)
direct_chain = direct_prompt | llm.with_structured_output(Answer, include_raw=True)

In [ ]:
run_model(direct_chain, test_data, direct_prompt, f'{folder_path}results/', 'direct_sample')

100%|██████████| 1273/1273 [12:16<00:00,  1.73it/s]

Results exported to /content/drive/MyDrive/IA024A_Processamento_de_Linguagem_Natural/Projeto Final/projeto/entrega4/results/direct_sample_20241112_105810.txt


In [ ]:
direct_answers, _ = parse_answers_from_raw_output(f'{folder_path}results/direct_sample_20241112_105810.txt')

<ipython-input-22-99863013cccc>:16: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  recovered_results = loads(tmp[-1])[0]


In [ ]:
len(direct_answers)

1273

In [ ]:
accuracy_report(answers=direct_answers, dataset=test_data, out_path=f'{folder_path}results/', file_name='direct.csv')

Accuracy
|    | questions   |   accuracy |
|---:|:------------|-----------:|
|  0 | general     |   0.735271 |
|  1 | step1       |   0.729013 |
|  2 | step2&3     |   0.742424 |

Distribution of wrong answers
|    | predictions   |   0 |
|---:|:--------------|----:|
|  0 | A             |  93 |
|  1 | B             |  96 |
|  2 | C             |  77 |
|  3 | D             |  71 |


## 3 Chain-of-thought

In [ ]:
class AnswerCoT(BaseModel):
    ''' Your answer to the question.
    '''
    final_answer: Literal['A', 'B', 'C', 'D'] = Field(description='Final answer to the question')
    step_by_step: Optional[str] = Field(description='Step-by-step reasoning to answer the question', default=None)

In [ ]:
cot_prompt_template = '''
Answer the question below.

Question: {question}
Options: {options}

Answer: Let's think step-by-step...
'''.strip()

In [ ]:
cot_prompt = ChatPromptTemplate.from_template(cot_prompt_template)
cot_chain = cot_prompt | llm.with_structured_output(AnswerCoT, include_raw=True)

In [ ]:
run_model(cot_chain, test_data, cot_prompt, f'{folder_path}results/', 'cot_test')

100%|██████████| 1273/1273 [1:36:22<00:00,  4.54s/it]

Results exported to /content/drive/MyDrive/IA024A_Processamento_de_Linguagem_Natural/Projeto Final/projeto/entrega4/results/cot_test_20241112_111658.txt


In [22]:
cot_answers, cot_reasoning = parse_answers_from_raw_output(f'{folder_path}results/cot_test_20241112_111658.txt')

1273


In [24]:
len(cot_answers)

1273

In [26]:
accuracy_report(answers=cot_answers, dataset=test_data, out_path=f'{folder_path}results/', file_name='cot.csv', reasoning=cot_reasoning)

Accuracy
|    | questions   |   accuracy |
|---:|:------------|-----------:|
|  0 | general     |   0.738413 |
|  1 | step1       |   0.727541 |
|  2 | step2&3     |   0.750842 |

Distribution of wrong answers
|    | predictions   |   0 |
|---:|:--------------|----:|
|  0 | A             |  92 |
|  1 | B             |  91 |
|  2 | C             |  81 |
|  3 | D             |  69 |


## 4 Agents

### Document embedding and indexing

For experiments with different splitters, retriever methods and embedding models, see [pipipi](popopo)

In [ ]:
def create_vector_store(list_documents, text_splitter, embed_model, save_to_file=True, file_path='drive/MyDrive/ia024a/projeto/entrega4/retriever/', add_info=None):
    '''
    Splits and embeds a list of documents.
    '''

    # splits list of documents into chunks and adds the source txt filename as metadata
    split_documents = text_splitter.create_documents([doc.page_content for doc in list_documents], metadatas=[{'ref': doc.metadata['source'].split('/')[-1]} for doc in list_documents])

    # embeds and indexes each chunk using FAISS
    vector_store = FAISS.from_documents(split_documents, embed_model)

    if save_to_file:
        filename = file_path + 'faiss__' + add_info + '__' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        vector_store.save_local(filename)

    return vector_store

### Load retriever

In [12]:
# emb_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
emb_model_name = 'avsolatorio/GIST-small-Embedding-v0'

In [13]:
embedding_model = HuggingFaceBgeEmbeddings(
    model_name=emb_model_name,
    # model_kwargs={'device': 'cuda'},
    # encode_kwargs={'batch_size': 32},
    # show_progress = True
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
# load saved faiss indexes
faiss = FAISS.load_local(
    folder_path=f'{folder_path}retrievers/faiss__300_30__20241110_163802',
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

In [15]:
# create a search tool from the faiss retriever
tool_search_faiss = create_retriever_tool(
    retriever=faiss.as_retriever(search_kwargs={'k': 3}),
    name='search_docs',
    description='Searches the query in documents and returns the top-3 most relevant ones.',
)

In [16]:
# react prompt
react_prompt_template = '''
Solve the question answering task alternating between Question, Thought, Action, Input and Observation steps.
You only have access to the following tools: {tools}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do. Break the problem down into subproblems and smaller steps and decide which action to take.
Action: the action to take, must be one of [{tool_names}]. If no action is needed, return your Final answer instead.
Action Input: the input to the action
Observation: the output of the action.
... (this Thought/Action/Action Input/Observation can repeat any number of times)
Final Answer: A/B/C/D

Begin!
Question: {question}
Options: {options}
Thought: {agent_scratchpad}
'''.strip()

In [17]:
react_prompt = ChatPromptTemplate.from_template(react_prompt_template)

In [18]:
# create agent with tools
agent = create_react_agent(
    llm=llm,
    tools=[tool_search_faiss],
    prompt=react_prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=[tool_search_faiss],
    verbose=False,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
    max_iterations=5
)

In [19]:
run_model(agent_executor, test_data, react_prompt, f'{folder_path}results/', 'react_test')

100%|██████████| 1273/1273 [2:47:08<00:00,  7.88s/it]

Results exported to /content/drive/MyDrive/ia024a/projeto/entrega4/results/react_test_20241112_214626.txt


In [20]:
react_answers, react_reasoning = parse_answers_from_raw_output(f'{folder_path}results/react_test_20241112_214626.txt')

69 Agent stopped due to iteration limit or time limit.
98 Agent stopped due to iteration limit or time limit.
104 Agent stopped due to iteration limit or time limit.
112 Agent stopped due to iteration limit or time limit.
132 Agent stopped due to iteration limit or time limit.
160 Agent stopped due to iteration limit or time limit.
185 Agent stopped due to iteration limit or time limit.
196 Agent stopped due to iteration limit or time limit.
204 Agent stopped due to iteration limit or time limit.
208 None of the options provided (A/B/C/D) accurately describe the mechanism of action of DN501 as a protease inhibitor.
332 Agent stopped due to iteration limit or time limit.
334 Agent stopped due to iteration limit or time limit.
355 Agent stopped due to iteration limit or time limit.
358 Agent stopped due to iteration limit or time limit.
454 Agent stopped due to iteration limit or time limit.
518 Agent stopped due to iteration limit or time limit.
533 Agent stopped due to iteration limit 

<ipython-input-10-8132a9dae174>:16: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  recovered_results = loads(tmp[-1])[0]


In [21]:
accuracy_report(answers=react_answers, dataset=test_data, out_path=f'{folder_path}results/', file_name='react.csv', reasoning=react_reasoning)

Accuracy
|    | questions   |   accuracy |
|---:|:------------|-----------:|
|  0 | general     |   0.747054 |
|  1 | step1       |   0.758468 |
|  2 | step2&3     |   0.734007 |

Distribution of wrong answers
|    | predictions   |   0 |
|---:|:--------------|----:|
|  0 | A             |  72 |
|  1 | B             |  71 |
|  2 | C             |  68 |
|  3 | D             |  67 |
|  4 | unanswered    |  44 |
